In [1]:
# initial
import os
import json
import pandas as pd 
from lxml.etree import HTML 
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

types = pd.read_excel('Honeywell Obsolete Products.xlsx')
types = types['Infrared Sensor Obsolescence - Affected Part Numbers']
print(types[:5])

#伪装成 chrome的设置
dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap['phantomjs.page.settings.userAgent'] = ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36')

# phantomjs 伪装成 chrome
browser = webdriver.PhantomJS(executable_path=r'E:\py_wd\phantomjs-2.1.1-windows\bin\phantomjs.exe',
                             desired_capabilities = dcap, service_args = ['--ignore-ssl-errors=true'])
browser.implicitly_wait(10) # 睡眠10秒等待加载

0    HLC1395-001
1    HLC1395-002
2    HOA0149-001
3    HOA0149-101
4    HOA1180-001
Name: Infrared Sensor Obsolescence - Affected Part Numbers, dtype: object


/home/khoa/env/local/lib/python3.5/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


WebDriverException: Message: 'E:\py_wd\phantomjs-2.1.1-windows\bin\phantomjs.exe' executable needs to be in PATH. 


In [2]:
def get_data(pn):
    url = 'https://www.eciaauthorized.com/en/search/'+pn.encode('utf-8')
    print('url: {}'.format(url))
    dt = pd.DataFrame()
    
    browser.get(url)
    html = HTML(browser.page_source) # 网页源代码
    a = html.xpath('//script/text()') # 提取script里面的内容
    for i in range(len(a)):
        if 'SetCompletion' in a[i]:
            len1 = a[i].encode('utf-8').find('ExactResults')+16 
            len2 = a[i].encode('utf-8').find('DistributorStatus')-12
            s = '{'+a[i].encode('utf-8')[len1:len2]+'}' # 提取详细的字段
            d = json.loads(s)
            print('Name:{}'.format(d['Distributor']['DisplayName']))
            print('Prices:{}'.format(d['Prices']))
            print('')
            df = pd.DataFrame(d['Prices'], columns = ['FormattedQuantity', 'FormattedPrice','Quantity' ])
            if len(df) > 0:
                df['PartNumber'] = pn
                df['Distributior'] = d['Distributor']['DisplayName']
                dt.append(df, ignore_index = True)
    return dt

In [ ]:
df = pd.DataFrame()
for pn in types:
    df.append(get_data(pn), ignore_index = True)
writer = pd.ExcelWriter('result.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()